In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
food_df=pd.read_csv('food_df.csv', index_col=0)
menu_db=pd.read_excel('최종DB(라벨링,위경도포함).xlsx', index_col=0)

### 클래스 생성

In [3]:
class FoodInfo:
    def __init__(self, calorie, first_nut, second_nut):
        self.calorie = calorie
        self.first_nut = first_nut
        self.second_nut = second_nut
        
        self.sorted_food = food_df.loc[food_df['에너지(㎉)'] < self.calorie]
        
        # 선호하는 1순위 영양소(first_nut)와 2순위 영양소(second_nut)를 입력
        # 1: 탄수화물   2: 단백질   3: 지방
        if first_nut == 1:
            if second_nut == 2:
                self.sorted_food = self.sorted_food[(self.sorted_food['탄수화물(g)'] > self.sorted_food['단백질(g)'])
                                                    & (self.sorted_food['단백질(g)'] > self.sorted_food['지방(g)'])]
            elif second_nut == 3:
                self.sorted_food = self.sorted_food[(self.sorted_food['탄수화물(g)'] > self.sorted_food['지방(g)'])
                                                    & (self.sorted_food['지방(g)'] > self.sorted_food['단백질(g)'])]
        if first_nut == 2:
            if second_nut == 1:
                self.sorted_food = self.sorted_food[(self.sorted_food['단백질(g)'] > self.sorted_food['탄수화물(g)'])
                                                    & (self.sorted_food['탄수화물(g)'] > self.sorted_food['지방(g)'])]
            elif second_nut == 3:
                self.sorted_food = self.sorted_food[(self.sorted_food['단백질(g)'] > self.sorted_food['지방(g)'])
                                                    & (self.sorted_food['지방(g)'] > self.sorted_food['탄수화물(g)'])]
        if first_nut == 3:
            if second_nut == 1:
                self.sorted_food = self.sorted_food[(self.sorted_food['지방(g)'] > self.sorted_food['탄수화물(g)'])
                                                    & (self.sorted_food['탄수화물(g)'] > self.sorted_food['단백질(g)'])]
            elif second_nut == 2:
                self.sorted_food = self.sorted_food[(self.sorted_food['지방(g)'] > self.sorted_food['단백질(g)'])
                                                    & (self.sorted_food['단백질(g)'] > self.sorted_food['탄수화물(g)'])]                
    
    def get_info(self):
        return self.sorted_food
    
    def set_kwd(self, kwd):
        self.kwd = kwd  
        
    def get_market(self):
        '''
        #1
        key word와 매칭되는 메뉴를 가진 음식점의 정보 제공
        화면에 보여줄 때 영양정보, cid는 제외한 '음식점 정보와 가게 메뉴'만 제공
        '''
        self.market = menu_db.loc[menu_db['정답']==self.kwd]
        return self.market.iloc[:, 1:9].drop(columns = 'cid')
    
    def get_similar_food(self):
        '''
        #2
        사용자가 선택한 음식과 유사한 음식의 영양 정보를 제공
        '영양 구성'이 비슷한 음식을 매칭
        '''
        # 칼로리로 필터링 된 음식들의 영양 정보만 피처벡터화 하여 코사인 유사도 계산
        cosine_sim = cosine_similarity(self.sorted_food.loc[:,['에너지(㎉)','탄수화물(g)','단백질(g)','지방(g)','나트륨(㎎)']])
        self.cosine_sim_df = pd.DataFrame(cosine_sim, index = self.sorted_food['식품명'], columns = self.sorted_food['식품명'])
                                     
        # 코사인 유사도 값(0~1 사이의 값)이 높은 순으로 정렬
        # 가장높은 1은 소비자가 처음 선택한 음식이므로 제외, 상위 10개의 음식명을 유사한 음식으로 판단, 리스트로 저장
        self.similar_menu = self.cosine_sim_df[self.kwd].sort_values(ascending = False)[1:11].index.tolist()
        
        self.similar_menu_info = pd.DataFrame()

        for i in range(len(self.sorted_food)):
            if self.sorted_food.iloc[i]['식품명'] in self.similar_menu:
                self.similar_menu_info = self.similar_menu_info.append(pd.DataFrame([self.sorted_food.iloc[i]]))
                                     
        return self.similar_menu_info

### 인스턴스 생성

In [5]:
# 200kcal 이하, 탄단지 중에서 단백질함량이 제일 높고 탄수화물이 두번째로 높은 음식 필터링
sample=FoodInfo(200, 2, 1)

In [6]:
sample.get_info()

,상용제품,식품명,지역 / 제조사,식품대분류,식품상세분류,1회제공량,내용량_단위,에너지(㎉),탄수화물(g),단백질(g),지방(g),나트륨(㎎),100g(ml)당 에너지,100g(ml)당 탄수화물,100g(ml)당 단백질,100g(ml)당 지방,100g(ml)당 나트륨
28,품목대표,바지락조개국,전국(대표),국 및 탕류,어패류국.탕,550.0,g,157.00,10.10,22.00,3.10,1596.17,28.55,1.84,4.00,0.56,290.21
31,품목대표,소고기무국,전국(대표),국 및 탕류,육류국.탕,400.0,g,123.00,7.80,14.20,4.00,1067.02,30.75,1.95,3.55,1.00,266.76
32,품목대표,소고기미역국,전국(대표),국 및 탕류,육류국.탕,650.0,g,151.00,7.70,18.10,5.30,1908.91,23.23,1.18,2.78,0.82,293.68
35,품목대표,올갱이국,괴산,국 및 탕류,어패류국.탕,600.0,g,144.26,13.00,13.70,4.20,2017.16,24.04,2.17,2.28,0.70,336.19
39,품목대표,홍합미역국,전국(대표),국 및 탕류,어패류국.탕,650.0,g,167.00,11.60,19.70,4.70,2043.68,25.69,1.78,3.03,0.72,314.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3037,외식,북어 무국,식품의대표값,국 및 탕류,어패류국.탕,300.0,g,56.32,4.32,8.43,0.59,717.00,18.77,1.44,2.81,0.20,239.00
3038,외식,소고기 무국,식품의대표값,국 및 탕류,육류국.탕,287.0,g,46.50,3.42,6.49,0.77,226.87,16.20,1.19,2.26,0.27,79.05
3042,외식,닭곰탕,식품의대표값,국 및 탕류,육류국.탕,336.0,g,89.36,4.20,17.88,0.12,653.52,26.60,1.25,5.32,0.04,194.50
3044,외식,코다리 뿌리채소 들깨탕,식품의대표값,국 및 탕류,어패류국.탕,347.0,g,93.33,3.64,14.75,2.20,155.11,26.90,1.05,4.25,0.63,44.70


In [7]:
sample.set_kwd('닭곰탕')

In [8]:
sample.get_market()

,가게,평점,평점수,리뷰수,주소,영업시간,가게 메뉴
1125,옥동식 한남점,3.8,5,0,서울 용산구 한남대로20길 61-7,NaN,구엄닭곰탕
2459,청계손국시,4.5,5,7,서울 서초구 동산로 9-5,영업시간 매일 10:00 ~ 20:00 · 브레이크타임 매일 14:30 ~ 18:00,당귀닭곰탕(토종닭)
2790,아차산 닭한마리,5.0,6,9,서울 광진구 광장로1나길 13,매일 10:00 ~ 22:00,들깨닭곰탕
4558,꿈꾸는메밀,4.1,12,144,서울 용산구 한강대로23길 55,매일 11:00 ~ 21:00,메밀 만두 닭곰탕
4583,메밀공방,3.5,31,55,서울 용산구 한강대로23길 55 용산아이파크몰 7층,매일 11:00 ~ 21:00,메밀꿩만두 닭곰탕
4615,메밀공방,3.5,31,55,서울 용산구 한강대로23길 55 용산아이파크몰 7층,매일 11:00 ~ 21:00,메밀만두 닭곰탕
4669,메밀공방,3.5,31,55,서울 용산구 한강대로23길 55 용산아이파크몰 7층,매일 11:00 ~ 21:00,메밀쌀밥 닭곰탕
6086,토종삼계탕,3.8,9,11,서울 서초구 서초대로51길 13 신원빌딩 1층,NaN,버섯닭곰탕
7889,황기엄나무 닭곰탕닭한마리,3.0,1,0,서울 종로구 종로 403,NaN,산낙지닭곰탕
11938,공돌이부엌 압구정로데오점,5.0,2,33,서울 강남구 선릉로157길 25 2층,"영업시간 월,화,수,일 11:30 ~ 23:00",얼큰사골닭곰탕


In [9]:
sample.get_similar_food()

,상용제품,식품명,지역 / 제조사,식품대분류,식품상세분류,1회제공량,내용량_단위,에너지(㎉),탄수화물(g),단백질(g),지방(g),나트륨(㎎),100g(ml)당 에너지,100g(ml)당 탄수화물,100g(ml)당 단백질,100g(ml)당 지방,100g(ml)당 나트륨
2563,품목대표,닭개장,전국(대표),국 및 탕류,육류국.탕,700.0,g,142.98,9.88,19.17,2.97,1168.18,20.43,1.41,2.74,0.42,166.88
2580,품목대표,새우탕,전국(대표),국 및 탕류,어패류국.탕,500.0,g,163.10,10.00,25.84,2.19,1340.99,32.62,2.00,5.17,0.44,268.20
2598,품목대표,육개장,전국(대표),국 및 탕류,육류국.탕,440.0,g,130.33,6.54,16.21,4.37,884.24,29.62,1.49,3.68,0.99,200.96
2785,품목대표,잔멸치풋고추볶음,전국(대표),볶음류,어패류볶음,50.0,g,127.82,8.47,14.12,4.16,1035.08,255.64,16.94,28.24,8.32,2070.16
2793,품목대표,풋고추멸치볶음,전국(대표),볶음류,어패류볶음,50.0,g,94.83,6.65,10.31,3.00,672.44,189.66,13.30,20.62,6.00,1344.88
2917,품목대표,병어조림,전국(대표),조림류,어패류조림,100.0,g,112.38,4.87,12.46,4.78,842.21,112.38,4.87,12.46,4.78,842.21
2920,품목대표,쇠고기장조림,전국(대표),조림류,육류조림,100.0,g,68.76,4.84,10.41,0.86,543.47,68.76,4.84,10.41,0.86,543.47
2950,품목대표,동태찌개,전국(대표),찌개 및 전골류,어패류찌개.전골,200.0,g,87.22,3.44,12.22,2.73,546.61,43.61,1.72,6.11,1.36,273.30
2983,품목대표,꼬막찜,전국(대표),찜류,어패류찜,200.0,g,192.87,13.93,24.06,4.55,1472.19,96.44,6.96,12.03,2.28,736.10
2997,품목대표,코다리찜,전국(대표),찜류,어패류찜,200.0,g,188.69,12.15,28.97,2.69,1240.64,94.34,6.08,14.48,1.34,620.32


In [10]:
print(sample.similar_menu)

['꼬막찜', '쇠고기장조림', '풋고추멸치볶음', '병어조림', '육개장', '코다리찜', '새우탕', '닭개장', '잔멸치풋고추볶음', '동태찌개']
